In [0]:
pwd

'/content'

In [0]:
pip install --upgrade --pre SimpleITK --find-links https://github.com/SimpleITK/SimpleITK/releases/tag/latest

Looking in links: https://github.com/SimpleITK/SimpleITK/releases/tag/latest
     |████████████████████████████████| 43.3MB 89kB/s 


In [0]:
cd drive/My\ Drive/Gleason-master

/content/drive/My Drive/Gleason-master


In [0]:
print(tf.__version__)


2.2.0-rc1


In [0]:
import cv2

In [0]:
from tqdm import tqdm 


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import sys
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
import re
from PIL import Image

In [0]:
from keras.models import Model
from keras.layers import Input, BatchNormalization, Concatenate, PReLU, ELU
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
import numpy as np

Using TensorFlow backend.


In [0]:
#from tensorflow.keras.engine import Layer
from tensorflow.keras.applications.vgg16 import *
from tensorflow.keras.models import *
#from tensorflow.keras.applications.imagenet_utils import _obtain_input_shape
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D, Cropping2D, Conv2D
from tensorflow.keras.layers import Input, Add, Dropout, Permute, add
from tensorflow.compat.v1.layers import conv2d_transpose
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [0]:
from keras.utils import *

In [0]:
train_frame_path = '/content/drive/My Drive/Gleason-master/train_frames/train'

train_mask_path = '/content/drive/My Drive/Gleason-master/train_masks/train'

val_frame_path = '/content/drive/My Drive/Gleason-master/val_frames/val'
val_mask_path = '/content/drive/My Drive/Gleason-master/val_masks/val'


In [0]:
def data_gen(img_folder, mask_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    m = os.listdir(mask_folder) #List of training masks
    random.shuffle(n)
    
    while (True):
      img = np.zeros((batch_size, 512, 512, 3)).astype('float')
      mask = np.zeros((batch_size, 512, 512, 6)).astype('float')

      for i in range(c, c+batch_size): #initially from 0 to 16, c = 0. 
        train_img = cv2.imread(img_folder+'/'+n[i])/255.
        train_img =  cv2.resize(train_img, (512, 512))# Read an image from folder and resize
      
        img[i-c] = train_img #add to array - img[0], img[1], and so on.
                                                   

        train_mask = cv2.imread(mask_folder+'/'+m[i], cv2.IMREAD_GRAYSCALE)/255.
        train_mask = cv2.resize(train_mask, (512, 512))
        train_mask = train_mask.reshape(512, 512, 1) # Add extra dimension for parity with train_img size [512 * 512 * 3]
        train_mask = to_categorical(train_mask, 6)
    
        mask[i-c] = train_mask

      c = c+batch_size
      if(c+batch_size>=len(os.listdir(img_folder))):
        c=0
        random.shuffle(n)
      yield img, mask

In [0]:
# Train the model
train_gen = data_gen(train_frame_path,train_mask_path, batch_size = 4)
val_gen = data_gen(val_frame_path,val_mask_path, batch_size = 4)

## **Defining** U-Net model for semantic segmentation


---



In [0]:
def tversky_loss(y_true, y_pred):
  alpha = 0.5
  beta  = 0.5
    
  ones = K.ones(K.shape(y_true))
  p0 = y_pred      # proba that voxels are class i
  p1 = ones-y_pred # proba that voxels are not class i
  g0 = y_true
  g1 = ones-y_true
    
  num = K.sum(p0*g0, (0,1,2,3))
  den = num + alpha*K.sum(p0*g1,(0,1,2,3)) + beta*K.sum(p1*g0,(0,1,2,3))
    
  T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    
  Ncl = K.cast(K.shape(y_true)[-1], 'float32')
  return Ncl-T

In [0]:
def dice_coef(y_true, y_pred):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
  return 1.-dice_coef(y_true, y_pred)

In [0]:
import tensorflow as tf


IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 3

batch_shape=(512,512,3)
#inputs = Input(batch_shape=(4, 512, 512, 3))
#Build the model
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
#s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(6, (1, 1), activation='softmax')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 512, 512, 16) 448         input_4[0][0]                    
__________________________________________________________________________________________________
dropout_27 (Dropout)            (None, 512, 512, 16) 0           conv2d_57[0][0]                  
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 512, 512, 16) 2320        dropout_27[0][0]                 
____________________________________________________________________________________________

# Define model callback settings


In [0]:
#tb = TensorBoard(log_dir='logs', write_graph=True)
#mc = ModelCheckpoint(mode='max', filepath='prostate.h5', monitor='accuracy', save_best_only='True', save_weights_only='True', verbose=1)
#es = EarlyStopping(mode='max', monitor='val_accuracy', patience=10, verbose=1)
#callbacks = [tb, mc, es]

In [0]:
no_of_training_images = 200
no_of_validation_images = 44


In [0]:
batch_size = 4


results = model.fit_generator(train_gen, epochs=5, 
                          steps_per_epoch = (no_of_training_images//batch_size),
                          validation_data=val_gen, 
                          validation_steps=(no_of_validation_images//batch_size), verbose=1)

Epoch 1/5
50/50 [==============================] - 1539s 31s/step - loss: 0.2199 - accuracy: 0.9127 - val_loss: 9.5001e-06 - val_accuracy: 1.0000
Epoch 2/5
50/50 [==============================] - 690s 14s/step - loss: 5.6109e-05 - accuracy: 1.0000 - val_loss: 1.1015e-05 - val_accuracy: 1.0000
Epoch 3/5
50/50 [==============================] - 683s 14s/step - loss: 2.9896e-05 - accuracy: 1.0000 - val_loss: 1.0951e-05 - val_accuracy: 1.0000
Epoch 4/5
50/50 [==============================] - 677s 14s/step - loss: 2.3671e-05 - accuracy: 1.0000 - val_loss: 5.8782e-06 - val_accuracy: 1.0000
Epoch 5/5
50/50 [==============================] - 680s 14s/step - loss: 1.6906e-05 - accuracy: 1.0000 - val_loss: 3.3387e-06 - val_accuracy: 1.0000


In [0]:
model.save('Model.h5')

In [0]:
pwd

'/content'